In [ ]:
!pip install virtualenv

In [ ]:
import virtualenv
import subprocess
import os
import sys

notebook_name = "anomaly"
virtualenv.cli_run(["venvs/" + notebook_name, "--no-download"])

venv_dir = "venvs/" + notebook_name
python_path = os.path.join(venv_dir, "bin", "python")
display_name = "Python (" + notebook_name + ")"
kernel_name = notebook_name

# Установка ipykernel в venv
subprocess.check_call([os.path.join(venv_dir, "bin", "pip"), "install", "ipykernel"])

# Регистрация ядра
subprocess.check_call([
    python_path, "-m", "ipykernel", "install",
    "--user",
    "--name", kernel_name,
    "--display-name", display_name
])

In [1]:
import sys
print(sys.executable)

/workspace/anomaly_detectors/venvs/anomaly/bin/python


In [ ]:
import subprocess
import os
import sys

# Путь к pip в активном ядре
pip_path = os.path.join(sys.prefix, "bin", "pip")

subprocess.check_call([pip_path, "install", "requests", "pyod", "matplotlib", "scikit-learn", "pandas", "numpy", "seaborn", "fedot", "fedot_ind", "giotto-tda", "ruptures"])

In [5]:
# Импорты
from scipy import stats

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import typing
from abc import abstractmethod, ABC
import math
from numpy import percentile
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from statsmodels.tsa.api import ExponentialSmoothing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import NearestNeighbors
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.mcd import MCD
from pyod.models.pca import PCA
from pyod.models.lscp import LSCP
from fedot_ind.api.main import FedotIndustrial
from fedot_ind.core.repository.constanst_repository import VALID_LINEAR_DETECTION_PIPELINE

import os
import time
import requests
import numpy as np
import statistics
import re
import json

ImportError: cannot import name 'VALID_LINEAR_DETECTION_PIPELINE' from 'fedot_ind.core.repository.constanst_repository' (/workspace/anomaly_detectors/venvs/anomaly/lib/python3.10/site-packages/fedot_ind/core/repository/constanst_repository.py)

In [ ]:
def load_data(data_path="../../data/series/"):
    """Загрузка временных рядов из CSV файлов"""
    data_dict = {}
    data_path = Path(data_path)
    
    if not data_path.exists():
        print(f"Папка {data_path} не найдена!")
        return data_dict
    
    for csv_file in data_path.glob("*.csv"):
        try:
            df = pd.read_csv(csv_file)
            if 'timestamp' in df.columns and 'close' in df.columns:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
                df = df.set_index('timestamp').sort_index()
                symbol = csv_file.stem
                data_dict[symbol] = df[['close']]
                print(f"Загружен {symbol}: {len(df)} точек")
            else:
                print(f"Пропущен {csv_file.name}: нет колонок 'timestamp' или 'close'")
        except Exception as e:
            print(f"Ошибка загрузки {csv_file.name}: {e}")
    
    return data_dict

# Загрузка данных
data = load_data()
print(f"\nЗагружено временных рядов: {len(data)}")

if len(data) > 0:
    # Выберем первый доступный ряд для демонстрации
    symbol = list(data.keys())[0]
    df = data[symbol]
    print(f"\nИспользуем ряд: {symbol}")
    print(f"Период: {df.index[0]} - {df.index[-1]}")
    print(f"Количество точек: {len(df)}")
    
    # Покажем статистику
    print("\nСтатистика ряда:")
    print(df['close'].describe())

# Генерация аномалий

## BoxPlot

In [ ]:
dp = df['PHOR'].copy()
dp.set_index('timestamp', inplace=True)
dp.head()

In [ ]:
# 2. Фильтрация по нужному интервалу
start_date = '2023-11-01'
end_date   = '2024-02-29'
mask = (dp.index >= start_date) & (dp.index <= end_date)
close_values = dp.loc[mask, 'close']

# 3. Построение горизонтального боксплота
plt.figure(figsize=(8, 4))
plt.boxplot(close_values, vert=False)
plt.title('Распределение цен закрытия (2023-11 – 2024-02)')
plt.xlabel('Цена закрытия')
plt.tight_layout()
plt.show()

## Определение распределения

In [ ]:
# 2. Shapiro–Wilk
stat_sw, p_sw = stats.shapiro(close_values)
print(f"Shapiro–Wilk: W={stat_sw:.4f}, p-value={p_sw:.4f}")

# 3. D’Agostino’s K²
stat_k2, p_k2 = stats.normaltest(close_values)
print(f"D’Agostino’s K²: χ²={stat_k2:.4f}, p-value={p_k2:.4f}")

# 4. Anderson–Darling
result_ad = stats.anderson(close_values, dist='norm')
print(f"Anderson–Darling: A²={result_ad.statistic:.4f}")
for sl, crit in zip(result_ad.significance_level, result_ad.critical_values):
    print(f"  {sl}% level: {crit:.4f}")

# 5. (Опционально) Lilliefors (в пакете statsmodels)
from statsmodels.stats.diagnostic import lilliefors
stat_l, p_l = lilliefors(close_values, dist='norm')
print(f"Lilliefors: D={stat_l:.4f}, p-value={p_l:.4f}")

alpha = 0.05  # уровень значимости
ad_crit_5 = result_ad.critical_values[result_ad.significance_level.tolist().index(5.0)]
if (p_sw > alpha and p_k2 > alpha and p_l > alpha and result_ad.statistic < ad_crit_5):
    print("\nИтог: нет оснований отвергать нормальность — распределение близко к нормальному.")
else:
    print("\nИтог: есть основания полагать, что распределение отличается от нормального.")

## Генерация данных из распределеня

In [ ]:
original = close_values.copy()

# 2. Оценка параметров
mu = original.mean()
sigma = original.std(ddof=0)  # МЛ-оценка σ

# 3. Генерация синтетических данных
n = len(original)
synthetic = np.random.normal(loc=mu, scale=sigma, size=n)

# 4. Объединение в один «двойной» ряд
combined = pd.Series(
    np.concatenate([original.values, synthetic]),
    index=pd.date_range(
        start=original.index[0],
        periods=2*n,
        freq=original.index.freq or 'D'
    ),
    name='close'
)

# (Опционально) Визуализация
plt.figure(figsize=(10,4))
plt.plot(combined, label='Original + Synthetic')
plt.axvline(combined.index[n], color='red', linestyle='--', label='Переход к синтетике')
plt.legend()
plt.title('Удвоенный ряд: реальные и сгенерированные значения')
plt.xlabel('Дата')
plt.ylabel('Цена закрытия')
plt.tight_layout()
plt.show()

## Генерацция данных из доходностей

In [ ]:
prices = close_values.copy()

# 2. Перевод в лог-доходности
log_returns = np.log(prices).diff().dropna()

# 3. Оценка параметров
mu_r = log_returns.mean()
sigma_r = log_returns.std(ddof=0)

# 4. Генерация синтетических доходностей и реконструкция цен
n = len(log_returns)
simulated_r = np.random.normal(loc=mu_r, scale=sigma_r, size=n)
P0 = prices.iloc[0]
simulated_prices = P0 * np.exp(np.cumsum(simulated_r))

# 5. Визуализация
plt.figure(figsize=(10, 4))
plt.plot(prices.index, prices.values, label='Реальные цены')
plt.plot(prices.index[1:], simulated_prices, label='Симуляция по лог-доходностям')
plt.legend()
plt.title('Сравнение реального пути и синтетического на базе лог-доходностей')
plt.xlabel('Дата')
plt.ylabel('Цена закрытия')
plt.tight_layout()
plt.show()

## Генерация автоэнкодером

In [ ]:
series = close_values.copy()

# 2. Нормировка в [0,1]
scaler = MinMaxScaler()
series_scaled = scaler.fit_transform(series.values.reshape(-1, 1))

# 3. Формирование обучающих окон
L = 20  # длина скользящего окна
X = []
for i in range(len(series_scaled) - L + 1):
    X.append(series_scaled[i:i+L])
X = np.array(X)  # форма (n_samples, L, 1)

# 4. Определение архитектуры автоэнкодера
input_seq = keras.Input(shape=(L, 1))

# Кодировщик
x = layers.Flatten()(input_seq)
x = layers.Dense(16, activation='relu')(x)
encoded = layers.Dense(8, activation='relu', name='bottleneck')(x)

# Декодировщик
x = layers.Dense(16, activation='relu')(encoded)
x = layers.Dense(L, activation='sigmoid')(x)
decoded = layers.Reshape((L, 1))(x)

autoencoder = keras.Model(input_seq, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# 5. Обучение
history = autoencoder.fit(
    X, X,
    epochs=100,
    batch_size=16,
    validation_split=0.1,
    verbose=0
)

# 6. Восстановление всего ряда
#   Подготовим все окна, пропустим через автоэнкодер и "склеим" обратно
reconstructed = autoencoder.predict(X, verbose=0)
# Для каждого шага i берем только последнюю точку из окна i
rec_points = [reconstructed[i, -1, 0] for i in range(len(reconstructed))]
# Переводим из скейлинга обратно в оригинальные значения
rec_series = scaler.inverse_transform(np.array(rec_points).reshape(-1,1)).flatten()

# Так как у нас n_points = len(series)-L+1, подставим соответствующий шаг
time_index = series.index[L-1:L-1+len(rec_series)]

# 7. Визуализация
plt.figure(figsize=(10,4))
plt.plot(series.index, series.values, label='Оригинал')
plt.plot(time_index, rec_series, label='Восстановлено автоэнкодером')
plt.title('Оригинальный ряд и его восстановление автоэнкодером')
plt.xlabel('Дата')
plt.ylabel('Цена закрытия')
plt.legend()
plt.tight_layout()
plt.show()


## Замешивание аномалий

In [ ]:
def inject_anomalies(
    series: pd.Series,
    n: int,
    anomaly_timestamps: list[pd.Timestamp] = None,
    anomaly_prices: list[float] = None,
    sigma_multiplier: float = 3.0,
    random_state: int = None
) -> tuple[pd.Series, pd.DataFrame]:
    """
    Вставляет n аномалий в pandas.Series с datetime-индексом.

    Параметры:
    - series: Series с индексом типа datetime и значениями цен
    - n:      число аномалий
    - anomaly_timestamps: необязательный список из n меток времени для ручной вставки
    - anomaly_prices:     необязательный список из n цен аномалий
    - sigma_multiplier:   множитель локальной волатильности для автогенерации
    - random_state:       seed для reproducibility

    Возвращает:
    - series_out: Series той же длины, с «вкрапленными» аномалиями
    - injected_df: DataFrame с колонками
        ['timestamp','original_price','anomaly_price']
    """
    if not isinstance(series, pd.Series):
        raise ValueError("Ожидается pandas.Series")
    if n <= 0:
        raise ValueError("n должно быть положительным целым")

    # подготовка
    rng = np.random.default_rng(random_state)
    series = series.sort_index().copy()

    # определяем, где вставлять аномалии
    if anomaly_timestamps is not None or anomaly_prices is not None:
        # ручная вставка
        if anomaly_timestamps is None or anomaly_prices is None:
            raise ValueError("Нужны оба аргумента: anomaly_timestamps и anomaly_prices")
        if len(anomaly_timestamps) != n or len(anomaly_prices) != n:
            raise ValueError("anomaly_timestamps и anomaly_prices длины n")

        # Ensure timestamps are timezone-naive if the series index is
        idxs_raw = pd.to_datetime(anomaly_timestamps)
        if series.index.tz is None and idxs_raw.tz is not None:
             idxs = idxs_raw.tz_convert(None)
        elif series.index.tz is not None and idxs_raw.tz is None:
             raise TypeError("Series index is timezone-aware, but anomaly_timestamps are naive.")
        else:
             idxs = idxs_raw # Both are naive or both are aware

        # Convert to the exact type/frequency of the series index if necessary
        # This step might be crucial for exact matches in the index
        idxs = pd.DatetimeIndex(idxs, dtype=series.index.dtype, freq=series.index.freq)

        for t in idxs:
            if t not in series.index:
                raise KeyError(f"Серии не содержит метку {t}")
        anomaly_map = dict(zip(idxs, anomaly_prices))

    else:
        # автоматическая вставка
        # Select from the index values and convert back to DatetimeIndex
        # to ensure compatibility for .loc lookup
        idxs = pd.DatetimeIndex(rng.choice(series.index.values, size=n, replace=False),
                                dtype=series.index.dtype, freq=series.index.freq)

        # оценка локальной волатильности через std разностей
        sigma = series.diff().std(skipna=True)
        noise = rng.normal(0, sigma * sigma_multiplier, size=n)
        anomaly_map = {t: series.loc[t] + noise[i] for i, t in enumerate(idxs)}

    # внесём аномалии и соберём информацию
    injected = []
    series_out = series.copy()
    for t in idxs:
        orig = series_out.loc[t]
        anom = anomaly_map[t]
        series_out.loc[t] = anom
        injected.append({
            "timestamp": t,
            "original_price": orig,
            "anomaly_price": anom
        })

    injected_df = pd.DataFrame(injected).set_index("timestamp")
    return series_out, injected_df

In [ ]:
# 1. Берём реальный обрезанный ряд
prices = close_values.copy()  # ваша Series из предыдущего шага

# 2. Переводим в лог-доходности и генерим синтетику
log_returns = np.log(prices).diff().dropna()
mu_r = log_returns.mean()
sigma_r = log_returns.std(ddof=0)
simulated_r = np.random.default_rng(42).normal(loc=mu_r, scale=sigma_r, size=len(log_returns))
P0 = prices.iloc[0]
simulated_prices = pd.Series(
    P0 * np.exp(np.cumsum(simulated_r)),
    index=prices.index[1:]
)

# 3. Вставляем в синтетику n=5 аномалий автоматически
augmented, anomalies = inject_anomalies(
    series=simulated_prices,
    n=5,
    sigma_multiplier=4.0,
    random_state=123
)

# 4. Визуализируем
plt.figure(figsize=(10,4))
plt.plot(simulated_prices.index, simulated_prices.values, label='Симуляция без аномалий')
plt.plot(augmented.index, augmented.values, label='С аномалиями', alpha=0.8)
plt.scatter(anomalies.index, anomalies['anomaly_price'],
            color='red', label='Аномалии', zorder=5)
plt.legend()
plt.title('Смесь синтетического ряда и вкрапленных аномалий')
plt.xlabel('Дата')
plt.ylabel('Цена')
plt.tight_layout()
plt.show()

In [ ]:
# 1. Выбираем вручную, в какие даты и на какие уровни вставить аномалии
manual_ts = [
    simulated_prices.index[4],  # 5th timestamp (index 4)
    simulated_prices.index[9],  # 10th timestamp (index 9)
    simulated_prices.index[14], # 15th timestamp (index 14)
]

# Пусть мы хотим, чтобы цена в эти даты прыгнула на +200, -150 и +300 пунктов соответственно
manual_prices = [
    simulated_prices.loc[manual_ts[0]] * 1.05, #
    simulated_prices.loc[manual_ts[1]] * 0.95, #
    simulated_prices.loc[manual_ts[2]] * 1.1, #
]

# 2. Вызываем функцию inject_anomalies с ручными параметрами
augmented_manual, anomalies_manual = inject_anomalies(
    series=simulated_prices,
    n=3,
    anomaly_timestamps=manual_ts,
    anomaly_prices=manual_prices
)

# 3. Смотрим, что получилось
print("Таблица вручную вставленных аномалий:")
print(anomalies_manual)

# 4. Визуализируем оригинал и вручную заданные аномалии
plt.figure(figsize=(10,4))
plt.plot(simulated_prices.index, simulated_prices.values, label='Симуляция без аномалий')
plt.plot(augmented_manual.index, augmented_manual.values, label='С ручными аномалиями', alpha=0.8)
plt.scatter(
    anomalies_manual.index,
    anomalies_manual['anomaly_price'],
    color='red',
    s=50,
    label='Ручные аномалии',
    zorder=5
)
plt.legend()
plt.title('Ручная вставка аномалий в синтетический ряд')
plt.xlabel('Дата')
plt.ylabel('Цена')
plt.tight_layout()
plt.show()

## Статистические детекторы аномалий

In [ ]:
dd = df['MOEX'].copy()
dd['timestamp'] = pd.to_datetime(dd['timestamp'])
dd.set_index('timestamp', inplace=True)
dd.head()

In [ ]:
outliers_fraction = 0.2
column = 'close'
c = 'close'

In [ ]:
class AbstractAnomalyDetector(ABC):
    @abstractmethod
    def __init__(self, data: pd.Series, interval=None):
        """
        Detect anomalies and get label for each anomaly
        :param data: datetime indexed pd.Series
        :param interval: pair of dates to search for anomalies between
        """
        self.data = data
        if interval:
            self.start, self.end = interval
            if not self.start:
                self.start = 0
            if not self.end:
                self.end = self.data.shape[0]
        else:
            self.start, self.end = 0, self.data.shape[0]
        pass

    @abstractmethod
    def get_labels(self) -> pd.Series:
        pass


class OutlierDetector(AbstractAnomalyDetector):
    """
    Detects anomalies using distribution of data
    Detects:
            Outliers - such points where values are not in 3-sigma range of distribution
                (in other words values are too big or too low than the rest of the data);
    """

    def __init__(self, data: pd.Series, interval=None):
        super().__init__(data, interval)

    def get_labels(self):
        result = self._search_for_anomalies()
        result = result[~np.isnan(result)]
        result = pd.DataFrame(index=result.index, data={'label': result.values})['label']
        if self.start and self.end:
            result = result[self.start:self.end]
        return result

    def _search_for_anomalies(self):
        sigma_min, sigma_max = self.__get_stat()
        return self.data.apply(lambda r: 1 if r > sigma_max else -1 if r < sigma_min else np.nan)

    def __get_stat(self):
        mean_val = self.data.mean()
        std_val = self.data.std()
        sigma_min = mean_val - 3 * std_val
        sigma_max = mean_val + 3 * std_val
        return sigma_min, sigma_max


class AbstractDistributionBasedAnomalyDetector(AbstractAnomalyDetector):

    def __init__(self, data, interval: typing.Tuple[str, str] = None, prev_only: bool = False):
        super().__init__(data, interval)
        self.prev_only = prev_only

    def get_labels(self) -> pd.Series:
        result = self._search_for_anomalies()
        result = pd.DataFrame(index=self.data.index, data={'label': result})['label']
        result = result[~np.isnan(result)]
        return result

    @abstractmethod
    def _search_for_anomalies(self):
        pass


class RareDistributionDetector(AbstractDistributionBasedAnomalyDetector):
    """
    Detects anomalies using distribution of data
    Detects:
            Rare distributions zones -
                such zones where set of values consecutively falls out of range of n*sigma distributions;
    """

    def __init__(self, data, interval: typing.Tuple[str, str] = None, prev_only: bool = False, n=1, window=50):
        super().__init__(data, interval, prev_only)
        self.window = window
        if window > self.end - self.start:
            raise ValueError('Window is bigger than interval')
        if not 0 < n < 4:
            raise ValueError('n must be in (0,4)')
        self.n = n

    def _search_for_anomalies(self):
        length, = self.data.shape
        self.__get_stats(length)
        result = np.array([np.nan] * length)

        for i in range(self.window if self.prev_only else self.start, self.end - self.window + 1):
            is_anomaly = True
            if self.prev_only:
                self.__get_stats(i)
            for w in range(i, i + self.window):
                if is_anomaly:
                    diff = abs(self.data[w] - self.mean)
                    is_anomaly = self.sigma_min < diff < self.sigma_max
            if is_anomaly:
                result[i:i + self.window] = 1
        return result

    def __get_stats(self, i):
        self.mean = self.data[:i].mean()
        self.std = self.data[:i].std()
        self.sigma_min = (self.n - 1) * self.std
        self.sigma_max = self.sigma_min + self.std


class MeanAnomalyDetector(AbstractDistributionBasedAnomalyDetector):
    """
    Detects anomalies using distribution of data
    Detects:
            out-of-mean anomalies - such zones where values are bigger or lower than mean of given data;
    """

    def __init__(self, data, interval: typing.Tuple[str, str] = None, prev_only: bool = False, lower=True, window=50):
        super().__init__(data, interval, prev_only)
        self.window = window
        self.lower = lower
        if window > self.end - self.start:
            raise ValueError('Window is bigger than interval')

    def _search_for_anomalies(self):
        length, = self.data.shape
        self.__get_stats(length)
        result = np.array([np.nan] * length)

        for i in range(self.window if self.prev_only else self.start, self.end - self.window + 1):
            is_anomaly = True
            if self.prev_only:
                self.__get_stats(i)
            for w in range(i, i + self.window):
                if is_anomaly:
                    is_anomaly = self.data[w] < self.mean if self.lower \
                        else self.data[w] > self.mean
            if is_anomaly:
                result[i:i + self.window] = 1
        return result

    def __get_stats(self, i):
        self.mean = self.data[:i].mean()


class DistributionBasedAnomalyDetector(AbstractDistributionBasedAnomalyDetector):
    """
        Detects anomalies using distribution of data
        Detects:
                distributions-change zones -
                    such zones where normal distribution of given data changes;
    """

    def __init__(self, data, interval: typing.Tuple[str, str] = None, prev_only: bool = False, threshold=0.3,
                 window=50):
        super().__init__(data, interval, prev_only)
        self.window = window
        if window > self.end - self.start:
            raise ValueError('Window is bigger than interval')
        self.threshold = threshold

    def _search_for_anomalies(self):
        length, = self.data.shape
        result = np.array([np.nan] * length)
        std_val = self.data.std()
        mean_val = self.data.mean()

        for i in range(self.window if self.prev_only else self.start, self.end - self.window + 1):
            data_slice = self.data[i:i + self.window]
            sl_mean = data_slice.mean()
            sl_std = data_slice.std()
            if self.prev_only:
                prev_slice = self.data[:i]
                mean_val = prev_slice.mean()
                std_val = prev_slice.std()
            is_anomaly = (sl_std > (1 + self.threshold) * std_val or sl_std < (1 - self.threshold) * std_val) \
                         and (sl_mean > (1 + self.threshold) * mean_val or sl_mean < (1 - self.threshold) * mean_val)
            if is_anomaly:
                result[i:i + self.window] = 1
        return result

In [ ]:
detectors = {
    'outlier_detector': OutlierDetector(dd[column]),
    'great_of_mean_detector': MeanAnomalyDetector(dd[column], lower=False, window=14),
    'least_of_mean_detector': MeanAnomalyDetector(dd[column], lower=True, window=14),
    'distribution_based_detector': DistributionBasedAnomalyDetector(dd[column]),
    'rare_1_distribution_detector': RareDistributionDetector(dd[column], n=1, window=25),
    'rare_2_distribution_detector': RareDistributionDetector(dd[column], n=2, window=7),
    'rare_3_distribution_detector': RareDistributionDetector(dd[column], n=3, window=3)
}

for detector_name, detector in detectors.items():
  result = detector.get_labels()

  if result.shape[0] > 0 :
      res = pd.DataFrame(result)
      dd[detector_name] = res['label']

      a = dd.loc[~dd[detector_name].isna(), [column]]
      fig, ax = plt.subplots(figsize=(16,9))

      ax.plot(dd[column])
      ax.scatter(a.index, a, c='red')

      plt.title(detector_name)
      plt.show()

## Прогнозный подход

### EMA

In [ ]:
outliers_fraction = 0.05

In [ ]:
df_test = dd[-50:]
df_train = dd[:-50]

In [ ]:
data = df_train[c]
data.index = pd.DatetimeIndex(data.index.values)
data

In [ ]:
fit = ExponentialSmoothing(data, seasonal_periods=5, trend='add', seasonal='add').fit()

In [ ]:
forecast = fit.forecast(50).to_frame().reset_index()
forecast.head()

In [ ]:
datetime_index_for_forecast = df_test.index
forecast.index = datetime_index_for_forecast
forecast = forecast.rename(columns={0: 'forecast'})
forecast = forecast.drop(columns=['index'])
print(forecast.head())

In [ ]:
fig, ax =plt.subplots()
forecast.plot(ax=ax)
df_test['close'].plot(ax=ax)
plt.show()

In [ ]:
df_res = forecast.copy()
df_res['fact'] = df_test[c]
df_res['MAPE'] = abs(df_res.fact - df_res.forecast) / df_res.fact
df_res.head()

In [ ]:
threshold = percentile(df_res.MAPE, 90)
df_res['anomaly'] = df_res.MAPE > threshold
df_res.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
a = df_res.loc[df_res.anomaly, ['fact']]

ax.plot(df_res.index, df_res.fact)
ax.scatter(a.index, a.fact, color='red')
plt.show()

## Isolation Forest

In [ ]:
scaler = StandardScaler()
np_scaled = scaler.fit_transform(dd['close'].values.reshape(-1,1))
data = pd.DataFrame(np_scaled)

data.head()

In [ ]:
dd['anomaly'] = model.predict(data)
dd.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
a = dd.loc[dd.anomaly == -1, [c]]

ax.plot(dd.index, dd[c])
ax.scatter(a.index, a[c], color='red')
plt.show()

## Кластеризация

In [ ]:
dd = df['MOEX']
dd = dd.set_index('timestamp')
dd.head()

### K-means

#### Elbow Cirve

In [ ]:
n_cls = range(1, 20)

kmeans = [KMeans(n_clusters=i).fit(dd) for i in n_cls]
scores = [kmeans[i].score(dd) for i in range(len(kmeans))]

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(n_cls, scores)
plt.show()

#### PCA

In [ ]:
X = dd.values
X_std = StandardScaler().fit_transform(X)

In [ ]:
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

In [ ]:
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i]) for i in range(len(eig_vals))]
eig_pairs.sort(key=lambda x: x[0], reverse=True)

tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [ ]:
plt.figure(figsize=(16,9))
plt.bar(range(len(var_exp)), var_exp)
plt.step(range(len(var_exp)), cum_var_exp)
plt.show()

#### K-MEANS

In [ ]:
def getDistanceByPoint(data, model):
    distance = pd.Series()
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.at[i]=np.linalg.norm(Xa-Xb)
    return distance

In [ ]:
data = pd.DataFrame(X_std)

In [ ]:
pca = PCA(n_components=2)
data = pca.fit_transform(data)

scaler = StandardScaler()
np_scaled = scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
data.head()

In [ ]:
kmeans_model = KMeans(n_clusters=7).fit(data)

In [ ]:
dd.reset_index(inplace=True)
dd['cluster'] = kmeans_model.predict(data)
dd.index = data.index
dd['pca1'] = data[0]
dd['pca2'] = data[1]

dist = getDistanceByPoint(data, kmeans_model)
n_outliers = int(0.05*len(dist))
threshold = percentile(dist, 95)
dd['anomaly_cls'] = (dist >= threshold).astype(int)
dd.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

a = dd.loc[dd.anomaly_cls == 1, [c, 'timestamp']]

ax.plot(dd.timestamp, dd[c])
ax.scatter(a.timestamp, a[c], color='red')
plt.show()

### K-NN

In [ ]:
knn = NearestNeighbors(n_neighbors=3)
knn.fit(X)

In [ ]:
dist, idxs = knn.kneighbors(X)

In [ ]:
dd['dist'] = dist.mean(axis=1)
threshold = percentile(dd.dist, 95)

dd['anomaly_knn'] = dd.dist > threshold
dd.head()

In [ ]:
a = dd.loc[dd['anomaly_knn'], [c, 'timestamp']]

fig, ax = plt.subplots(figsize=(16, 9))

ax.plot(dd.timestamp, dd[c])
ax.scatter(a.timestamp, a[c], color='red')
plt.show()

## Многомерные аномалии (PyOD)

In [ ]:
def show_plots(df, columns, anomaly_column=None):
    fig, axs = plt.subplots(len(columns), 1, sharex=True, constrained_layout=True, figsize=(12,10))
    for i in range(len(columns)):
        c = columns[i]

        axs[i].plot(dd.index, df[c], color='gray',label='Normal')

        if anomaly_column:
            a = df.loc[df[anomaly_column] == 1, [c]] #anomaly
            axs[i].scatter(a.index, a[c], color='red', label='Anomaly')

        axs[i].xaxis_date()
        axs[i].set_title(c)
        plt.xlabel('Date')
    plt.show()

In [ ]:
dd = df['MOEX']
dd = dd.set_index('timestamp')
dd.head()

In [ ]:
columns = ['open', 'close', 'high', 'low', 'volume']
show_plots(dd, columns)

### HBOS

In [ ]:
outliers_fraction = 0.05
random_state = np.random.RandomState(42)

In [ ]:
detector = HBOS(contamination=outliers_fraction)

In [ ]:
X = dd.values

detector.fit(X)

pred = detector.predict(X)

dd['anomaly'] = pred
show_plots(dd, columns, 'anomaly')

### ABOD

In [ ]:
outliers_fraction = 0.05
random_state = np.random.RandomState(42)

In [ ]:
detector = ABOD(contamination=outliers_fraction)

In [ ]:
X = dd.values

detector.fit(X)

pred = detector.predict(X)
dd['anomaly'] = pred

show_plots(dd, columns, 'anomaly')

### LOF

In [ ]:
dd = df['MOEX']
dd = dd.set_index('timestamp')
dd.head()

In [ ]:
outliers_fraction = 0.05
random_state = np.random.RandomState(42)

In [ ]:
lof_detector = LOF(n_neighbors=20, contamination=outliers_fraction)

In [ ]:
X = dd.values

lof_detector.fit(X)

pred = lof_detector.predict(X)

dd['anomaly'] = pred

show_plots(dd, columns, 'anomaly')

### CBLOF

In [ ]:
cblof_detector = CBLOF(contamination=outliers_fraction, random_state=random_state)

In [ ]:
X = dd.values

cblof_detector.fit(X)

pred = cblof_detector.predict(X)

dd['anomaly'] = pred

show_plots(dd, columns, 'anomaly')

### Isolation Forest

In [ ]:
dd = df['MOEX']
dd = dd.set_index('timestamp')
dd.head()

In [ ]:
outliers_fraction = 0.05
random_state = np.random.RandomState(42)

In [ ]:
if_detector = IForest(contamination=outliers_fraction, random_state=random_state)

In [ ]:
X = dd.values

if_detector.fit(X)
pred = if_detector.predict(X)
dd['anomaly'] = pred

show_plots(dd, columns, 'anomaly')

### KNN

In [ ]:
knn_detector = KNN(contamination=outliers_fraction, method='mean')

In [ ]:
X = dd.values

knn_detector.fit(X)
pred = knn_detector.predict(X)
dd['anomaly'] = pred

show_plots(dd, columns, 'anomaly')

### MCD

In [ ]:
mcd_detector = MCD(contamination=outliers_fraction, random_state=random_state)

In [ ]:
X = dd.values

mcd_detector.fit(X)

pred = mcd_detector.predict(X)

dd['anomaly'] = pred

show_plots(dd, columns, 'anomaly')

### PCA

In [ ]:
pca_detector = PCA(contamination=outliers_fraction, random_state=random_state)

In [ ]:
X = dd.values
pca_detector.fit(X)

pred = pca_detector.predict(X)

dd['anomaly'] = pred

show_plots(dd, columns, 'anomaly')

### LSCP

In [ ]:
outliers_fraction = 0.05
random_state = np.random.RandomState(42)

In [ ]:
detector_list = [
    PCA(contamination=outliers_fraction, random_state=random_state),
    HBOS(contamination=outliers_fraction),
    LOF(contamination=outliers_fraction),
    ABOD(contamination=outliers_fraction),
    IForest(contamination=outliers_fraction, random_state=random_state)
]

lscp_detector = LSCP(detector_list, contamination=outliers_fraction, random_state=random_state)

In [ ]:
X = dd.values
lscp_detector.fit(X)

pred = lscp_detector.predict(X)

dd['anomaly'] = pred

show_plots(dd, columns, 'anomaly')

## FEDOT

In [ ]:
# 1. Подготовка ряда
series = df['MOEX'].set_index('timestamp')['close']
# Разобьём на train/test (80%/20%), чтобы не «подглядывать» в будущие точки
split_idx = int(len(series) * 0.8)
train_series = series.iloc[:split_idx]
test_series  = series.iloc[split_idx:]

# Упакуем в формат, который ждёт FedotIndustrial:
#   для anomaly_detection y не обязателен, поэтому передаём лишь X
train_data = (train_series.values.reshape(-1, 1), None)
test_data  = (test_series.values.reshape(-1, 1), None)

# 2. Список встроенных линейных детекторов
pipeline_labels = [
    'stat_detector',
    'arima_detector',
    'iforest_detector',
    'conv_ae_detector',
    'lstm_ae_detector'
]

# 3. Для каждого детектора: обучаем, предсказываем и рисуем
fig, axes = plt.subplots(len(pipeline_labels), 1, figsize=(12, 4 * len(pipeline_labels)), sharex=True)

for ax, label in zip(axes, pipeline_labels):
    # 3.1 Инициализируем AutoML-сервис на конкретном детекторе
    ad = FedotIndustrial(
        problem='classification',
        industrial_strategy='anomaly_detection',
        industrial_task_params={
            'detection_window': 10,
            'detection_pipeline': label  # указываем нужный детектор
        },
        timeout=2,    # минут
        n_jobs=1,
        logging_level=30
    )

    # 3.2 Обучаем на train_series
    ad.fit(train_data)
    # 3.3 Предсказываем на test_series
    labels = ad.predict(test_data).ravel()  # 0/1

    # 3.4 Находим timestamps аномалий
    anomaly_idx = test_series.index[labels == 1]
    anomaly_vals = test_series.loc[anomaly_idx]

    # 3.5 Рисуем
    ax.plot(test_series.index, test_series.values, label='test series')
    ax.scatter(anomaly_idx, anomaly_vals, color='red', s=40, label='anomaly')
    ax.set_title(f"{label} (n_anomalies={len(anomaly_idx)})")
    ax.set_ylabel("Цена")
    ax.legend()

axes[-1].set_xlabel("Дата")
plt.tight_layout()
plt.show()

In [ ]:
from fedot_ind.api.main import FedotIndustrial

# 1. Подготовка multivariate-рядов
#    Используем все числовые колонки MOEX
df_moex = df['MOEX'].copy().set_index('timestamp')
features = ['open','high','low','close','volume']
X = df_moex[features].values
times = df_moex.index

# 2. Train/Test split (80%/20%)
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
times_test = times[split:]

train_data = (X_train, None)
test_data  = (X_test, None)

# 3. Список линейных детекторов
pipeline_labels = [
    'stat_detector',
    'arima_detector',
    'iforest_detector',
    'conv_ae_detector',
    'lstm_ae_detector'
]

# 4. Прогоняем каждый детектор и сохраняем метки
anomalies_idx = {}

for label in pipeline_labels:
    ad = FedotIndustrial(
        problem='classification',
        industrial_strategy='anomaly_detection',
        industrial_task_params={
            'detection_window': 10,
            'detection_pipeline': label
        },
        timeout=2,    # минуты
        n_jobs=1,
        logging_level=30
    )
    ad.fit(train_data)
    labels = ad.predict(test_data).ravel().astype(bool)  # True = аномалия
    anomalies_idx[label] = times_test[labels]

# 5. Визуализация: пять графиков для close
fig, axes = plt.subplots(len(pipeline_labels), 1,
                         figsize=(12, 3 * len(pipeline_labels)),
                         sharex=True)

for ax, label in zip(axes, pipeline_labels):
    ax.plot(times_test, X_test[:, features.index('close')],
            label='close')
    ax.scatter(anomalies_idx[label],
               df_moex.loc[anomalies_idx[label], 'close'],
               color='red', s=40, label='anomaly', zorder=5)
    ax.set_title(f"{label} — найдено {len(anomalies_idx[label])} аномалий")
    ax.set_ylabel("Цена закрытия")
    ax.legend()

axes[-1].set_xlabel("Дата")
plt.tight_layout()
plt.show()


In [ ]:
## TODO OPENROUTER

# Ваш токен доступа к LLM-прокси
bearer_token = os.environ.get("BEARER_TOKEN") or ""
API_SEND   = "https://api.gen-api.ru/api/v1/networks/gpt-4o-mini"
API_STATUS = "https://api.gen-api.ru/api/v1/request/get/{}"

HEADERS = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'Bearer {bearer_token}'
}

def extract_indices(result_data):
    """
    Извлекает список индексов из ответа LLM, убирая любые
    ```code fences``` и лишние символы.
    """
    # 1) Приводим к строке
    text = result_data if isinstance(result_data, str) else str(result_data)

    # 2) Убираем все «```json» или «```» в начале и в конце
    #    - убираем ```json\n или ```\n
    text = re.sub(r"^```(?:json)?\s*\n?", "", text)
    #    - убираем заключительные ```
    text = re.sub(r"\n?```$", "", text)

    # 3) Находим первый JSON-массив вида [..]
    match = re.search(r"\[.*?\]", text, flags=re.DOTALL)
    if not match:
        return []

    arr_text = match.group(0)
    try:
        return json.loads(arr_text)
    except json.JSONDecodeError:
        # пытаемся почистить пробелы и кавычки
        arr_text = re.sub(r"[\"']", "", arr_text)
        nums = re.findall(r"\d+", arr_text)
        return [int(x) for x in nums]

def compute_patch_stats(patch):
    ex = float(np.mean(patch))
    dx = float(np.std(patch))
    try:
        mode_val = float(statistics.mode([round(x,2) for x in patch]))
    except statistics.StatisticsError:
        mode_val = None
    pct = (patch[-1] - patch[0]) / patch[0] * 100
    return ex, dx, mode_val, pct

def make_patch_messages(window, patch_size=5):
    msgs = []
    n_patches = len(window) // patch_size
    for i in range(n_patches):
        p = window[i*patch_size:(i+1)*patch_size]
        ex, dx, mode_val, pct = compute_patch_stats(p)
        nums = ", ".join(f"{x:.2f}" for x in p)
        hint = (
            f"min: {min(p):.2f}, max: {max(p):.2f}, "
            f"mean: {ex:.2f}, std: {dx:.2f}, change%: {pct:+.2f}%"
        )
        content = (
            f"Patch {i+1} (indices {i*patch_size}-{(i+1)*patch_size-1}): "
            f"[{nums}]\nHint — {hint}"
        )
        msgs.append({"role": "system", "content": content})
    return msgs

def send_request(messages):
    resp = requests.post(API_SEND, headers=HEADERS, json={"messages": messages})
    resp.raise_for_status()
    return resp.json()["request_id"]

def wait_for_result(request_id, timeout=30):
    url = API_STATUS.format(request_id)
    start = time.time()
    while True:
        r = requests.get(url, headers={"Authorization": f"Bearer {bearer_token}"})
        r.raise_for_status()
        data = r.json()
        if data.get("status") == "success":
            return data["result"]
        if time.time() - start > timeout:
            raise TimeoutError("LLM request timed out")
        time.sleep(0.5)

# === AD Detection with window step = W/2 ===

prices = list(df['MOEX']['close'][-40:])  # последние 40 точек
W = 30                                  # размер окна
step = W // 2                           # шаг = половина окна
patch_size = 5

all_anomalies = []

for start in range(0, len(prices) - W + 1, step):
    window = prices[start:start + W]

    # 1) Патчи и статистика
    msgs = make_patch_messages(window, patch_size)

    # 2) Общий prompt
    msgs.insert(0, {
        "role": "system",
        "content": (
            "You are an anomaly detection assistant. "
            "Given 30 consecutive closing prices broken into patches with statistics, "
            "identify which individual price indices in the window are anomalies.  "
            "Return **only** a JSON array of zero-based indices relative to the window "
            "where anomalies occur (e.g. [2, 17, 29]). "
            "If there are no anomalies, return an empty array: []."
        )
    })

    # 3) User prompt
    msgs.append({
        "role": "user",
        "content": "Detect anomalous price indices in this window."
    })

    # 4) Запрос к LLM
    try:
        req_id = send_request(msgs)
        result_data = wait_for_result(req_id)

        indices = extract_indices(result_data)

    except Exception as e:
        print(f"Error at window starting {start}: {e}")
        indices = []

    # переводим в глобальные индексы
    global_idxs = [start + idx for idx in indices]
    all_anomalies.append(global_idxs)
    print(f"Window {start}-{start+W-1}: anomalies at {global_idxs}")

print("\nAll detected anomalies (global indices):")
print(all_anomalies)

# Поиск точек перелома

## Fedot

In [ ]:
# 1. Подготовим данные
series = df['MOEX'].set_index('timestamp')['close']
# FedotIndustrial ожидает X как массив shape (n_samples, n_features)
X = series.values.reshape(-1, 1)

# 2. Инициализируем FedotIndustrial для каждого детектора
pipelines = {
    'StatDetector': 'stat_detector',
    'ARIMA_Detector': 'arima_detector'
}

# Будем хранить для каждого детектора индексы changepoints
changepoints = {}

for name, pipeline_label in pipelines.items():
    ad = FedotIndustrial(
        problem='classification',            # бинарная задача: 0/1
        industrial_strategy='anomaly_detection',
        industrial_task_params={
            'detection_window': 10,          # окно для оценки изменений
            'detection_pipeline': pipeline_label
        },
        timeout=1,                           # время AutoML в минутах
        n_jobs=1,
        logging_level=30
    )
    # 3. Обучаем на всём ряде
    ad.fit((X, None))
    # 4. Предсказываем метки змей: 1 означает «anomaly»/changepoint
    labels = ad.predict((X, None)).ravel().astype(bool)
    # 5. Сохраняем временные метки точек перелома
    changepoints[name] = series.index[labels]

# 6. Визуализируем оба результата
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
for ax, (name, idxs) in zip(axes, changepoints.items()):
    ax.plot(series.index, series.values, label='close')
    ax.scatter(idxs, series.loc[idxs], color='red', s=50,
               label='changepoint', zorder=5)
    ax.set_title(f"Точки перелома ({name}) — найдено {len(idxs)}")
    ax.set_ylabel("Цена закрытия")
    ax.legend()

axes[-1].set_xlabel("Дата")
plt.tight_layout()
plt.show()

## Ruptures

In [ ]:
import pandas as pd
import ruptures as rpt
import matplotlib.pyplot as plt

# 1. Загрузка ряда
# Предполагаем, что у вас уже есть DataFrame df['MOEX'] с колонками ['timestamp','close']
series = df['MOEX'].set_index('timestamp')['close'].sort_index()
signal = series.values  # numpy array

# 2. Выбор алгоритма и поиск точек перелома
#    Здесь — PELT с RBF-моделью и penalization=10 (можно подбирать)
algo = rpt.Pelt(model="rbf").fit(signal)
breakpoints = algo.predict(pen=10)  # возвращает список индексов после которых меняется поведение

print("Найденные точки перелома (индексы после смены сегмента):", breakpoints)

# 3. Визуализация результатов
plt.figure(figsize=(12, 6))
# Removed the 'show=True' argument
rpt.display(signal, breakpoints)
plt.title("Change Point Detection — PELT (model='rbf')")
plt.xlabel("Шаг (нулевой отсчёт)")
plt.ylabel("Цена закрытия")
# Explicitly call plt.show() to display the plot
plt.show()